In [39]:
%matplotlib widget
import pandas as pd
import numpy as np
import csv 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.widgets import TextBox
import matplotlib
import math
import sklearn.model_selection
from sklearn.neighbors import KNeighborsRegressor
import metamodell
from IPython.display import display
import customstats

# Read CSV-Data

In [40]:
def load():
    filename="collected_results.csv"
    with open(filename,"r") as fil:
        fields=[i.strip() for i in fil.readline().split(",")]
    data=np.genfromtxt(filename,skip_header=1,delimiter=",")
    return fields,data
fields,data=load()

# Select outputs to train modell for

In [41]:
desired_outputs={'error_schicht','error_wez0', 'error_delr0', 'error_wez90', 'error_delr90', 'error_wez', 'error_delr', 'error_ges'}
nr_inputs=21
#do not use energy-ratio since entirely dependent on relative expansion
desired_idx=[i for i in range(nr_inputs) if fields[i] != "energy_ratio"]+[i for i,name in enumerate(fields) if name in desired_outputs]
nr_inputs=nr_inputs-1
fields,data=load()
data=data[:,desired_idx]
metamodell_fields=[fields[i] for i in desired_idx]

# generate Test- and Train-Data

In [42]:
nr_points=data.shape[0]
ratio_test=0.25 #how many datapoints shall be used for testing
idx_train,idx_test=sklearn.model_selection.train_test_split(range(nr_points),test_size=ratio_test)


# define kinds of meta_modells to train

In [43]:
Regressors=[]
Regressors.append(("nearest_neighbour",KNeighborsRegressor()))

# evaluate different metamodells

In [44]:

def evaluate_row(row_predicted,row_real):
    #error when predicting the resulting error directly
    errors=[0.0 for i in range(3)]
    error_ges_real=row_real['error_ges']
    error_ges_predicted=row_predicted['error_ges']
    errors[0]=customstats.rel_deviation(error_ges_real,error_ges_predicted)**2
    
    #error when predicting the errors for wez,cut and schicht
    error_wez_predicted=row_predicted['error_wez']
    error_spalt_predicted=row_predicted['error_delr']
    error_schicht_predicted=row_predicted['error_schicht']
    error_ges_predicted=customstats.sqrt_MSE([error_wez_predicted,error_spalt_predicted])+error_schicht_predicted
    errors[1]=customstats.rel_deviation(error_ges_real,error_ges_predicted)**2
    
    #error when predicting the errors for wez,cut for each qs and schicht
    errors_qs_predicted=[row_predicted['error_wez0'],row_predicted['error_wez90'],row_predicted['error_delr0'],row_predicted['error_delr90']]
    error_ges_predicted=customstats.sqrt_MSE(errors_qs_predicted)+error_schicht_predicted
    errors[2]=customstats.rel_deviation(error_ges_real,error_ges_predicted)**2
    return tuple(errors)
modells=[]
names=[]
#create modells
for name,Regressor in Regressors:
    modell=metamodell.metamodell(data,Regressor,nr_inputs,evaluate_row,metamodell_fields)
    modell.split_data(idx_train,idx_test)
    modell.train()
    modells.append(modell)
    names.append(name)
#evaluate modells
nr_modells=len(names)
nr_errors=3
for reduce_function in [np.amax,np.average]:
    array_modell_error=np.zeros((nr_modells,nr_errors))
    df_modell_error=pd.DataFrame(array_modell_error,names)
    print(reduce_function.__name__)
    for name,modell in zip(names,modells):
        df_modell_error.loc[name]=modell.test(reduce_function)
    display(df_modell_error)

amax


,0,1,2
nearest_neighbour,0.230389,0.230701,0.230865


average


,0,1,2
nearest_neighbour,0.042798,0.042672,0.042655
